In [1]:
from config import opt
import numpy as np
from lib.res_model import RES18_SSD, RES101_SSD
from lib.vgg_model import VGG_SSD
from lib.resnet import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from lib.utils import detection_collate

from lib.multibox_encoder import MultiBoxEncoder
from lib.ssd_loss import MultiBoxLoss

from voc_dataset import VOCDetection

In [2]:
 model = RES18_SSD(opt.num_classes, opt.anchor_num, pretrain=True)

resnet18 pretrain_model loading...


In [13]:
param_list = list(model.named_parameters())

In [24]:
print(param_list[2][1].shape)

torch.Size([64])


In [37]:
bias_tensors_meta = []
bias_tensors = []

for j in range(len(param_list)):
    if "bias" in param_list[j][0]:
        bias_tensors_meta.append(param_list[j][0])
        bias_tensors.append(param_list[j][1].detach().numpy())
        name = bias_tensors_meta[-1]
        print(name, bias_tensors[-1].shape)
        np.save(name + "_bias.npy",bias_tensors[-1].squeeze())
        print(bias_tensors[-1].squeeze().shape)

res.1.bias (64,)
(64,)
res.4.0.bn1.bias (64,)
(64,)
res.4.0.bn2.bias (64,)
(64,)
res.4.1.bn1.bias (64,)
(64,)
res.4.1.bn2.bias (64,)
(64,)
res.5.0.bn1.bias (128,)
(128,)
res.5.0.bn2.bias (128,)
(128,)
res.5.0.downsample.1.bias (128,)
(128,)
res.5.1.bn1.bias (128,)
(128,)
res.5.1.bn2.bias (128,)
(128,)
res.6.0.bn1.bias (256,)
(256,)
res.6.0.bn2.bias (256,)
(256,)
res.6.0.downsample.1.bias (256,)
(256,)
res.6.1.bn1.bias (256,)
(256,)
res.6.1.bn2.bias (256,)
(256,)
res.7.0.bn1.bias (512,)
(512,)
res.7.0.bn2.bias (512,)
(512,)
res.7.0.downsample.1.bias (512,)
(512,)
res.7.1.bn1.bias (512,)
(512,)
res.7.1.bn2.bias (512,)
(512,)
res.9.bias (1024,)
(1024,)
res.11.bias (1024,)
(1024,)
extras.0.bias (256,)
(256,)
extras.1.bias (512,)
(512,)
extras.2.bias (128,)
(128,)
extras.3.bias (256,)
(256,)
extras.4.bias (128,)
(128,)
extras.5.bias (256,)
(256,)
extras.6.bias (128,)
(128,)
extras.7.bias (256,)
(256,)
loc.0.bias (16,)
(16,)
loc.1.bias (24,)
(24,)
loc.2.bias (24,)
(24,)
loc.3.bias (24,)
(24,

In [31]:
weight_tensors_meta = []
weight_tensors = []


In [38]:
for j in range(len(param_list)):
    if "weight" in param_list[j][0]:
        weight_tensors_meta.append(param_list[j][0])
        weight_tensors.append(param_list[j][1].detach().numpy())
        name = weight_tensors_meta[-1]
        print(name, weight_tensors[-1].shape)
        
        np.save(name + ".npy",weight_tensors[-1].squeeze())
        np.save(name+"_transposed.npy",weight_tensors[-1].squeeze().transpose().copy())

L2Norm.weight (128,)
res.0.weight (64, 3, 7, 7)
res.1.weight (64,)
res.4.0.conv1.weight (64, 64, 3, 3)
res.4.0.bn1.weight (64,)
res.4.0.conv2.weight (64, 64, 3, 3)
res.4.0.bn2.weight (64,)
res.4.1.conv1.weight (64, 64, 3, 3)
res.4.1.bn1.weight (64,)
res.4.1.conv2.weight (64, 64, 3, 3)
res.4.1.bn2.weight (64,)
res.5.0.conv1.weight (128, 64, 3, 3)
res.5.0.bn1.weight (128,)
res.5.0.conv2.weight (128, 128, 3, 3)
res.5.0.bn2.weight (128,)
res.5.0.downsample.0.weight (128, 64, 1, 1)
res.5.0.downsample.1.weight (128,)
res.5.1.conv1.weight (128, 128, 3, 3)
res.5.1.bn1.weight (128,)
res.5.1.conv2.weight (128, 128, 3, 3)
res.5.1.bn2.weight (128,)
res.6.0.conv1.weight (256, 128, 3, 3)
res.6.0.bn1.weight (256,)
res.6.0.conv2.weight (256, 256, 3, 3)
res.6.0.bn2.weight (256,)
res.6.0.downsample.0.weight (256, 128, 1, 1)
res.6.0.downsample.1.weight (256,)
res.6.1.conv1.weight (256, 256, 3, 3)
res.6.1.bn1.weight (256,)
res.6.1.conv2.weight (256, 256, 3, 3)
res.6.1.bn2.weight (256,)
res.7.0.conv1.weigh

In [41]:
a = weight_tensors[-10]

In [45]:
a

array([[[[-3.82715371e-03,  1.11785689e-02, -1.36038344e-02],
         [-3.47360317e-03,  4.16658819e-04,  4.61961050e-03],
         [-4.87160496e-03,  1.00874761e-02,  1.00807054e-02]],

        [[-3.53723951e-03, -1.80446450e-03,  1.00523261e-02],
         [ 7.34674279e-03,  5.94232138e-03,  2.42562499e-03],
         [-1.41111035e-02, -9.83301084e-03, -9.74200480e-03]],

        [[ 1.16345445e-02,  8.48280732e-03, -2.54029967e-03],
         [-4.22163028e-03, -2.98376195e-03, -6.66606706e-03],
         [-3.17973085e-04,  3.60308308e-03, -8.39574262e-04]],

        ...,

        [[-8.74907151e-03,  6.61179516e-03,  1.25708720e-02],
         [-1.30286589e-02,  8.78848042e-03, -1.34869535e-02],
         [-1.46466028e-02,  7.27768335e-03, -1.43461395e-03]],

        [[ 7.74879474e-03, -1.36244623e-02,  5.53277228e-03],
         [ 1.46746030e-02, -4.78614494e-03,  8.27641133e-03],
         [-9.64196026e-03,  4.48799599e-03,  1.15613388e-02]],

        [[-3.03370412e-03, -6.02409616e-03, -8

In [44]:
np.sum(a != 0) / (24 * 512)

9.0